# Practice on Keras Functional API

In [1]:
# imports
import keras
from keras import layers
from keras import backend as K
import numpy as np

In [2]:
# mnist dataset
# loading training MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# reshaping and normalizing the images
x_train = x_train.reshape(60000, 784).astype('float32')/255  
x_test = x_test.reshape(10000, 784).astype('float32')/255
# one-hot encoding of labels
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)


In [3]:
# building the model
def build_model():
    inputs = keras.Input(shape=(784,), name='input')  # input layer
    x = layers.Dense(64, activation='relu', name='hidden1')(inputs)  # hidden layer
    x = layers.Dense(64, activation='relu', name='hidden2')(x)  # hidden layer
    outputs = layers.Dense(10, activation='softmax', name='output')(x)  # output layer
    return keras.Model(inputs=inputs, outputs=outputs)

model_ce = build_model()
model_fl = build_model()
model_rl = build_model()

model_rl.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 784)]             0         
                                                                 
 hidden1 (Dense)             (None, 64)                50240     
                                                                 
 hidden2 (Dense)             (None, 64)                4160      
                                                                 
 output (Dense)              (None, 10)                650       
                                                                 
Total params: 55050 (215.04 KB)
Trainable params: 55050 (215.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
optimizer = keras.optimizers.SGD()
ce_loss = keras.losses.CategoricalCrossentropy()
metrics = [keras.metrics.CategoricalAccuracy()]

In [5]:
# compiling the model
model_ce.compile(optimizer=optimizer, loss=ce_loss, metrics=metrics)

In [6]:
# training the model
model_ce.fit(x_train, y_train, batch_size=64, epochs=6, validation_split=0.2)

Epoch 1/6
750/750 [==============================] - 2s 2ms/step - loss: 1.1273 - categorical_accuracy: 0.7125 - val_loss: 0.4749 - val_categorical_accuracy: 0.8783
Epoch 2/6
750/750 [==============================] - 1s 2ms/step - loss: 0.4201 - categorical_accuracy: 0.8838 - val_loss: 0.3400 - val_categorical_accuracy: 0.9058
Epoch 3/6
750/750 [==============================] - 1s 2ms/step - loss: 0.3410 - categorical_accuracy: 0.9023 - val_loss: 0.3004 - val_categorical_accuracy: 0.9138
Epoch 4/6
750/750 [==============================] - 1s 2ms/step - loss: 0.3050 - categorical_accuracy: 0.9128 - val_loss: 0.2768 - val_categorical_accuracy: 0.9206
Epoch 5/6
750/750 [==============================] - 1s 2ms/step - loss: 0.2811 - categorical_accuracy: 0.9198 - val_loss: 0.2604 - val_categorical_accuracy: 0.9251
Epoch 6/6
750/750 [==============================] - 1s 2ms/step - loss: 0.2622 - categorical_accuracy: 0.9254 - val_loss: 0.2430 - val_categorical_accuracy: 0.9299


In [7]:
# evaluating the model
test_scores = model_ce.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])


313/313 - 0s - loss: 0.2423 - categorical_accuracy: 0.9313 - 296ms/epoch - 944us/step
Test loss: 0.24232876300811768
Test accuracy: 0.9312999844551086


In [8]:
# saving the model
path = './weights/mnist_ce_model.keras'
model_ce.save(path)

In [9]:
focal_loss = keras.losses.CategoricalFocalCrossentropy()

In [10]:
# compiling the model
optimizer = keras.optimizers.SGD()
model_fl.compile(optimizer=optimizer, loss=focal_loss, metrics=metrics)

In [11]:
# training the model
model_fl.fit(x_train, y_train, batch_size=64, epochs=6, validation_split=0.2)

Epoch 1/6
750/750 [==============================] - 2s 2ms/step - loss: 0.3325 - categorical_accuracy: 0.5701 - val_loss: 0.1962 - val_categorical_accuracy: 0.7266
Epoch 2/6
750/750 [==============================] - 1s 2ms/step - loss: 0.1416 - categorical_accuracy: 0.7690 - val_loss: 0.0970 - val_categorical_accuracy: 0.8260
Epoch 3/6
750/750 [==============================] - 1s 2ms/step - loss: 0.0887 - categorical_accuracy: 0.8298 - val_loss: 0.0712 - val_categorical_accuracy: 0.8603
Epoch 4/6
750/750 [==============================] - 2s 3ms/step - loss: 0.0709 - categorical_accuracy: 0.8576 - val_loss: 0.0603 - val_categorical_accuracy: 0.8781
Epoch 5/6
750/750 [==============================] - 1s 2ms/step - loss: 0.0620 - categorical_accuracy: 0.8725 - val_loss: 0.0542 - val_categorical_accuracy: 0.8863
Epoch 6/6
750/750 [==============================] - 1s 2ms/step - loss: 0.0565 - categorical_accuracy: 0.8808 - val_loss: 0.0502 - val_categorical_accuracy: 0.8926


In [12]:
# evaluating the model
test_scores = model_fl.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

313/313 - 0s - loss: 0.0503 - categorical_accuracy: 0.8933 - 254ms/epoch - 812us/step
Test loss: 0.05033818632364273
Test accuracy: 0.8932999968528748


In [13]:
# saving the model
path = './weights/mnist_focal_model.keras'
model_fl.save(path)

In [27]:
# defining rational loss function
# RL(p_t) = - 1/p_t * log(p_t)
def rational_loss():
    """
    Rational Loss for multi-class classification, Keras style.
    RL(p_t) = - 1/p_t * log(p_t), where p_t is the probability associated with the true class.

    :return: A callable rational_loss_fixed(y_true, y_pred) to be used as a Keras loss function.
    """
    def rational_loss_fixed(y_true, y_pred):
        """
        The actual loss computation.
        :param y_true: Ground truth labels, shape of [batch_size, num_classes].
        :param y_pred: Predicted class probabilities, shape of [batch_size, num_classes].
        :return: A scalar representing the mean rational loss over the batch.
        """
        # Clip the prediction value to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)
        # Compute the Rational Loss
        loss = (-1 / y_pred) * cross_entropy
        # Average the loss over the batch
        return K.mean(K.sum(loss, axis=-1))

    return rational_loss_fixed


In [15]:
# compiling the model
rational_loss = rational_loss()
optimizer = keras.optimizers.SGD()
model_rl.compile(optimizer=optimizer, loss=rational_loss, metrics=metrics)

In [16]:
# training the model
history = model_rl.fit(x_train, y_train, batch_size=64, epochs=6, validation_split=0.2)

Epoch 1/6
750/750 [==============================] - 2s 2ms/step - loss: -144364432.0000 - categorical_accuracy: 0.2383 - val_loss: -144498704.0000 - val_categorical_accuracy: 0.1035
Epoch 2/6
750/750 [==============================] - 1s 2ms/step - loss: -144764128.0000 - categorical_accuracy: 0.1019 - val_loss: -144498704.0000 - val_categorical_accuracy: 0.1035
Epoch 3/6
750/750 [==============================] - 1s 2ms/step - loss: -144764048.0000 - categorical_accuracy: 0.1019 - val_loss: -144498704.0000 - val_categorical_accuracy: 0.1035
Epoch 4/6
750/750 [==============================] - 1s 2ms/step - loss: -144764080.0000 - categorical_accuracy: 0.1019 - val_loss: -144498704.0000 - val_categorical_accuracy: 0.1035
Epoch 5/6
750/750 [==============================] - 1s 2ms/step - loss: -144764016.0000 - categorical_accuracy: 0.1019 - val_loss: -144498704.0000 - val_categorical_accuracy: 0.1035
Epoch 6/6
750/750 [==============================] - 1s 2ms/step - loss: -144764208.0

In [17]:
# evaluating the model
test_scores = model_rl.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

313/313 - 0s - loss: -1.4490e+08 - categorical_accuracy: 0.1010 - 316ms/epoch - 1ms/step
Test loss: -144901712.0
Test accuracy: 0.10100000351667404


In [18]:
# saving the model
path = './weights/mnist_rational_model.keras'
model_rl.save(path)

In [19]:
# # defining focal  loss function
# # FL(p_t) = -alpha_t * (1-p_t)^gamma * log(p_t)
# def focal_loss(alpha=0.25, gamma=2.0):
#     """
#     Focal Loss, Keras styles
#     :param alpha: Weighting factor for the positive class, typically in the range [0, 1].
#     :param gamma: Focusing parameter to down-weight well-classified examples, typically in the range [0, 5].
#     :return: A callable focal_loss_fixed(y_true, y_pred) to be used as a Keras loss function.
#     """
#     def focal_loss_fixed(y_true, y_pred):
#         """
#         The actual loss computation.
#         :param y_true: Ground truth labels, shape of [batch_size, num_classes].
#         :param y_pred: Predicted class probabilities, shape of [batch_size, num_classes].
#         :return: 
#         """
#         # Clip the prediction value to prevent NaN's and Inf's
#         y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
#         # Calculate Cross Entropy
#         cross_entropy = -y_true * K.log(y_pred)
#         # Calculate Focal Loss
#         loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
#         # Compute mean loss in batch
#         return K.mean(K.sum(loss, axis=-1))
#     return focal_loss_fixed


In [28]:
# building the model
def build_model():
    inputs = keras.Input(shape=(784,), name='input')  # input layer
    x = layers.Dense(64, activation='relu', name='hidden1')(inputs)  # hidden layer
    x = layers.Dense(64, activation='relu', name='hidden2')(x)  # hidden layer
    outputs = layers.Dense(10, activation='softmax', name='output')(x)  # output layer
    return keras.Model(inputs=inputs, outputs=outputs)

model_ce = build_model()
model_fl = build_model()
model_rl = build_model()


In [29]:
# imbalance 
def create_imbalanced_data(x, y, imbalance_rate=0.5):
    indices_by_class = [np.where(y[:, d] == 1)[0] for d in range(10)]
    selected_indices = []
    
    for d in range(10):
        frequency = int(len(indices_by_class[d]) * (imbalance_rate ** d))
        selected_indices.extend(indices_by_class[d][:frequency])
    
    return x[selected_indices], y[selected_indices]

x_train_imbalanced, y_train_imbalanced = create_imbalanced_data(x_train, y_train)


In [30]:
model_ce.compile(optimizer=keras.optimizers.SGD(), loss=keras.losses.CategoricalCrossentropy(), metrics=[keras.metrics.CategoricalAccuracy()])
model_fl.compile(optimizer=keras.optimizers.SGD(), loss=keras.losses.CategoricalFocalCrossentropy(), metrics=[keras.metrics.CategoricalAccuracy()])
model_rl.compile(optimizer=keras.optimizers.SGD(), loss=rational_loss(), metrics=[keras.metrics.CategoricalAccuracy()])


In [32]:
print("Training on imbalanced data:")
print("Cross Entropy:")
model_ce.fit(x_train_imbalanced, y_train_imbalanced, epochs=10, batch_size=32)
print("Focal Loss:")
model_fl.fit(x_train_imbalanced, y_train_imbalanced, epochs=10, batch_size=32)
print("Rational Loss:")
model_rl.fit(x_train_imbalanced, y_train_imbalanced, epochs=10, batch_size=32)

Training on imbalanced data:
Cross Entropy:
Epoch 1/10
383/383 [==============================] - 1s 1ms/step - loss: 0.1137 - categorical_accuracy: 0.9675
Epoch 2/10
383/383 [==============================] - 1s 1ms/step - loss: 0.1084 - categorical_accuracy: 0.9694
Epoch 3/10
383/383 [==============================] - 1s 1ms/step - loss: 0.1048 - categorical_accuracy: 0.9695
Epoch 4/10
383/383 [==============================] - 1s 1ms/step - loss: 0.1008 - categorical_accuracy: 0.9707
Epoch 5/10
383/383 [==============================] - 1s 1ms/step - loss: 0.0973 - categorical_accuracy: 0.9714
Epoch 6/10
383/383 [==============================] - 1s 1ms/step - loss: 0.0939 - categorical_accuracy: 0.9716
Epoch 7/10
383/383 [==============================] - 1s 1ms/step - loss: 0.0904 - categorical_accuracy: 0.9732
Epoch 8/10
383/383 [==============================] - 1s 1ms/step - loss: 0.0873 - categorical_accuracy: 0.9735
Epoch 9/10
383/383 [==============================] - 1s 1ms

In [33]:
# Balanced data
print(model_ce.evaluate(x_test, y_test))
print(model_fl.evaluate(x_test, y_test))
print(model_rl.evaluate(x_test, y_test))

# Imbalanced data
x_test_imbalanced, y_test_imbalanced = create_imbalanced_data(x_test, y_test)
print(model_ce.evaluate(x_test_imbalanced, y_test_imbalanced))
print(model_fl.evaluate(x_test_imbalanced, y_test_imbalanced))
print(model_rl.evaluate(x_test_imbalanced, y_test_imbalanced))

313/313 [==============================] - 0s 1ms/step - loss: 1.3555 - categorical_accuracy: 0.6879
[1.35547935962677, 0.6879000067710876]
313/313 [==============================] - 0s 1ms/step - loss: 0.3461 - categorical_accuracy: 0.5707
[0.346059113740921, 0.5706999897956848]
313/313 [==============================] - 0s 1ms/step - loss: -144901712.0000 - categorical_accuracy: 0.1010
[-144901712.0, 0.10100000351667404]
64/64 [==============================] - 0s 1ms/step - loss: 0.1091 - categorical_accuracy: 0.9707
[0.10905645787715912, 0.9706601500511169]
64/64 [==============================] - 0s 1ms/step - loss: 0.0227 - categorical_accuracy: 0.9550
[0.022728214040398598, 0.9550122022628784]
64/64 [==============================] - 0s 1ms/step - loss: -151250064.0000 - categorical_accuracy: 0.0616
[-151250064.0, 0.06161369010806084]


In [34]:
def accuracy_by_bins(model, x, y):
    predictions = model.predict(x).argmax(axis=-1)
    true_labels = y.argmax(axis=-1)
    bins = [(0, 1), (2, 7), (8, 9)]
    for bin_start, bin_end in bins:
        mask = (true_labels >= bin_start) & (true_labels <= bin_end)
        bin_accuracy = np.mean(predictions[mask] == true_labels[mask])
        print(f"Accuracy for bin {bin_start}-{bin_end}: {bin_accuracy}")

print("Accuracy by bins for balanced data:")
print("Cross Entropy:")
accuracy_by_bins(model_ce, x_test, y_test)
print("Focal Loss:")
accuracy_by_bins(model_fl, x_test, y_test)
print("Rational Loss:")
accuracy_by_bins(model_rl, x_test, y_test)

Accuracy by bins for balanced data:
Cross Entropy:
313/313 [==============================] - 0s 960us/step
Accuracy for bin 0-1: 0.9947990543735225
Accuracy for bin 2-7: 0.8053202304303626
Accuracy for bin 8-9: 0.011094301563287948
Focal Loss:
313/313 [==============================] - 0s 1ms/step
Accuracy for bin 0-1: 0.9947990543735225
Accuracy for bin 2-7: 0.6104710267705863
Accuracy for bin 8-9: 0.0
Rational Loss:
313/313 [==============================] - 0s 1ms/step
Accuracy for bin 0-1: 0.0
Accuracy for bin 2-7: 0.17112843104032532
Accuracy for bin 8-9: 0.0
